## clinical variable retrieval codebase.
each querey is referencing a sql querey on my computer for ALL patients in the database, then generating a dataframe, then paring that dataframe down to only the patients/icustay_id in our cohort. it's inefficient but works. 

In [11]:
#6-25-18
#the streamlined version of this notebook seeks to accomplish a few tasks:
#organize and standardize all sql code so that i am running a .sql file rather than typing sql code into jupyter
#change all filepaths to match the github linked directory to ensure better version control
###THIS IS STILL A WORK IN PROGRESS

time: 816 µs


In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
#patients of interest from rotation_cohort_generation

final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/16082018_final_pt_df2.csv', index_col=0)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 102 ms


## Access MIMIC database and convert it to dataframe in Pandas

In [3]:
import psycopg2
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password: ·····


time: 2.62 s


In [4]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

def sql_to_df_hadm(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['hadm_id'].isin(hadm_id)]
    return sql_exe_show_df

def clinvar_fxn(var_name, path):
    f= open(path, 'r')
    var = f.read()
    cur.execute('rollback')
    cur.execute(var)
    try:
        df= sql_to_df_icu(sql_exe_show('select * from %s;' %(var_name)))
    except KeyError or NameError:
        try:      
            df= sql_to_df_hadm(sql_exe_show('select * from %s;' %(var_name)))
        except KeyError or NameError:
            df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))

    print(df.shape)
    return(df)


time: 48.6 ms


# extracting clinical data for our patients

## Echodata in Noteevents

In [5]:
echodata_df= clinvar_fxn(
    'echodata',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/echodata.sql' 
) #took 34 seconds origionally. took 54 just now

(11494, 18)
time: 52.4 s


## Weight

In [19]:
# -- This query extracts weights for adult ICU patients on their first ICU day.
# -- It does *not* use any information after the first ICU day, as weight is
# -- sometimes used to monitor fluid balance.

weightfirstday_df= clinvar_fxn(
    'weightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/weightfirstday.sql' 
)


(12801, 6)
time: 7.19 s


## Height

In [7]:
# -- This query extracts heights for adult ICU patients.
# -- It uses all information from the patient's first ICU day.
# -- This is done for consistency with other queries - it's not necessarily needed.
# -- Height is unlikely to change throughout a patient's stay.
heightfirstday_df= clinvar_fxn(
    'heightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/heightfirstday.sql' 
)

(12801, 4)
time: 2.52 s


# Labs

In [8]:
# -- This query pivots lab values taken in the first 24 hours of a patient's stay
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit

labs_all_nosummary_df= clinvar_fxn(
    'labs_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/labs_all_nosummary.sql' 
)


(3036173, 6)
time: 6min 26s


## Vital Sign

In [9]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

vitals_all_nosummary_df= clinvar_fxn(
    'vitals_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/vitals_all_nosummary.sql' 
)

(14028103, 6)
time: 19min 44s


## Glasgow Coma Scale

In [10]:
gcsall_df= clinvar_fxn(
    'gcsall',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/gcsall.sql' 
)

(12801, 8)
time: 2min 13s


## Renal replacement therapy

In [11]:
rrt_all_df= clinvar_fxn(
    'rrt_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rrt_all.sql' 
)

(12801, 4)
time: 9.48 s


## Urine Output

In [12]:
urine_output_all_df= clinvar_fxn(
    'urine_output_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/urine_output_all.sql' 
)

(73302, 5)
time: 15.8 s


# UTI related variables

In [6]:
uti_all_df= clinvar_fxn(
    'uti_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/uti_all.sql' 
)

(645133, 12)
time: 15min 23s


In [10]:
uti_all_df.head(10)

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code
78,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,431
79,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,5070
80,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,4280
81,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,5849
82,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,2765
83,9,150750.0,51486,2149-11-09 11:32:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,4019
89,9,150750.0,51486,2149-11-10 09:24:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,431
90,9,150750.0,51486,2149-11-10 09:24:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,5070
91,9,150750.0,51486,2149-11-10 09:24:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,4280
92,9,150750.0,51486,2149-11-10 09:24:00,NEG,NaN,None,Leukocytes,Urine,Hematology,5799-2,5849


time: 25.2 ms


## Blood Gas Test

In [6]:
bg_all_nosummary_df= clinvar_fxn(
    'bg_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/bg_all_nosummary.sql' 
) #3min, 25 seconds origionally, took 14 minutes on friday...

(1884504, 6)
time: 14min 33s


In [7]:
bg_all_nosummary2_df= clinvar_fxn(
    'bg_all_nosummary2',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/bg_all_nosummary2.sql' 
)

(1884684, 7)
time: 17min 34s


In [ ]:
#my goal was to debug bg_all_nosummary2 and see how many rows we have that correspond to the vent count and catgories above, but the sql server is being really slow. 

In [8]:
bg_all_nosummary2_df

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value
607,9,150750,220597,2149-11-09 17:47:00,BASEEXCESS,1.00,1
608,9,150750,220597,2149-11-09 17:47:00,CHLORIDE,103.00,103
609,9,150750,220597,2149-11-09 17:47:00,FIO2,60.00,60
610,9,150750,220597,2149-11-09 17:47:00,GLUCOSE,178.00,178
611,9,150750,220597,2149-11-09 17:47:00,HEMATOCRIT,42.00,42
612,9,150750,220597,2149-11-09 17:47:00,HEMOGLOBIN,14.00,14.0
613,9,150750,220597,2149-11-09 17:47:00,INTUBATED,NaN,INTUBATED
614,9,150750,220597,2149-11-09 17:47:00,LACTATE,1.90,1.9
615,9,150750,220597,2149-11-09 17:47:00,PCO2,46.00,46
616,9,150750,220597,2149-11-09 17:47:00,PEEP,5.00,5


time: 176 ms


## Vasodil

In [27]:

weightdurations_df= clinvar_fxn(
    'weightdurations',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/weightdurations.sql'
)

epi_dose_df= clinvar_fxn(
    'epinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/epinephrine_dose.sql'
)
norepi_dose_df= clinvar_fxn(
    'norepinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/norepinephrine_dose.sql'
)
dopamine_dose_df= clinvar_fxn(
    'dopamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dopamine_dose.sql'
)
dobutamine_dose_df= clinvar_fxn(
    'dobutamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dobutamine_dose.sql'
)
vasopressin_dose_df= clinvar_fxn(
    'vasopressin_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/vasopressin_dose.sql'
)
phenylephrine_dose_df= clinvar_fxn(
    'phenylephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/phenylephrine_dose.sql'
)

(45448, 4)
(2397, 5)
(44371, 5)
(8900, 5)
(1922, 5)
(3472, 5)
(36608, 5)
time: 52.8 s


# ventilator settings and categorization

In [7]:
ventsettings_df= clinvar_fxn(
    'ventsettings',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/ventsettings.sql'
)
#going from wide format to long:
#pd.melt(ventsettings_df, id_vars=['icustay_id','charttime'])

(786605, 6)
time: 11min 39s


In [9]:

def vent_categorization(final_pt_df2, ventsettings_df):
    """
    overview: provides 3 variables, one for each 24 hour peroid following first ab administration, that codes vent status during that 24 hr peroid.
    ventilation data is based upon the ventsettings.sql file output. 
       categories: 
        - Vent- had mech vent within 24hr period
        - Oxygen- had only oxygentherapy within 24hr period
        - None- had neither within 24hr period.
        - Not_coded= data was not present
        
    input: final cohort dataframe (by icustay_id/hadm_id) annotated with t_0
    output: a dataframe with every icustay_id as a row, and a categorization for each of the first 3 days based on ventilation recieved:
    """
    
    ventsettings_merge_df= pd.merge(ventsettings_df,
                                final_pt_df2[['icustay_id','t_0']],
                                left_on='icustay_id',
                                right_on= 'icustay_id',
                                how='left')
 
    ventsettings_df_3dayannotation= final_pt_df2[['icustay_id','t_0']]
    ventsettings_df_3dayannotation['vent_day1']='Not_coded' #coding Not_coded as the default. must be sure to code over this to None to ensure I capture everyone
    ventsettings_df_3dayannotation['vent_day2']='Not_coded' #coding Not_coded as the default. must be sure to code over this to None to ensure I capture everyone
    ventsettings_df_3dayannotation['vent_day3']='Not_coded' #coding Not_coded as the default. must be sure to code over this to None to ensure I capture everyone
    ventsettings_df_3dayannotation.head()

    day1= (
        (pd.to_datetime(ventsettings_merge_df['t_0'])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('00:00:00','23:59:59'))
    day2= (
        (pd.to_datetime(ventsettings_merge_df['t_0'])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('24:00:00','47:59:59'))
    day3= (
        (pd.to_datetime(ventsettings_merge_df['t_0'])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('48:00:00','71:59:59'))

    mech_1= list(ventsettings_merge_df.loc[day1 & ventsettings_merge_df['mechvent']==1 ,'icustay_id'])
    mech_2= list(ventsettings_merge_df.loc[day2 & ventsettings_merge_df['mechvent']==1 ,'icustay_id'])
    mech_3= list(ventsettings_merge_df.loc[day3 & ventsettings_merge_df['mechvent']==1 ,'icustay_id'])
    O2_1= list(ventsettings_merge_df.loc[day1 & ventsettings_merge_df['oxygentherapy']==1 ,'icustay_id'])
    O2_2= list(ventsettings_merge_df.loc[day2 & ventsettings_merge_df['oxygentherapy']==1 ,'icustay_id'])
    O2_3= list(ventsettings_merge_df.loc[day3 & ventsettings_merge_df['oxygentherapy']==1 ,'icustay_id'])
    None_1= list(ventsettings_merge_df[(day1) & (ventsettings_merge_df['mechvent']==0) & (ventsettings_merge_df['oxygentherapy']==0) ]['icustay_id'])
    None_2= list(ventsettings_merge_df[(day2) & (ventsettings_merge_df['mechvent']==0) & (ventsettings_merge_df['oxygentherapy']==0) ]['icustay_id'])
    None_3= list(ventsettings_merge_df[(day3) & (ventsettings_merge_df['mechvent']==0) & (ventsettings_merge_df['oxygentherapy']==0) ]['icustay_id'])

    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(mech_1),'vent_day1']='Mech'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(mech_2),'vent_day2']='Mech'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(mech_3),'vent_day3']='Mech'

    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(O2_1),'vent_day1']='Oxygen'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(O2_2),'vent_day2']='Oxygen'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(O2_3),'vent_day3']='Oxygen'

    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(None_1),'vent_day1']='None'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(None_2),'vent_day2']='None'
    ventsettings_df_3dayannotation.loc[ventsettings_df_3dayannotation.loc[:,'icustay_id'].isin(None_3),'vent_day3']='None'

    return(ventsettings_df_3dayannotation)


time: 200 ms


In [ ]:
#the following code is an alternative method to vent_categorization fxn written above. the output gives the # of vent events in a given timeframe
#for each icustay_id
def vent_count(final_pt_df2,ventsettings_df):
    """
    overview: quantifies the # of ventilation events related to mechanical ventilation and oxygen therapy for each 24 hr window after first ab dose 
       categories: 
        - Vent- had mech vent within 24hr period
        - Oxygen- had only oxygentherapy within 24hr period
        
    input: final cohort dataframe (by icustay_id/hadm_id) annotated with t_0, ventsettings_df
    output: a dataframe with every icustay_id as a row, and a count for each of the first 3 days based on ventilation recieved:
    """
    ventsettings_merge_df= pd.merge(ventsettings_df,
                                    final_pt_df2[['icustay_id','t_0']],
                                    left_on='icustay_id',
                                    right_on= 'icustay_id',
                                    how='left')

    day1= (
        (pd.to_datetime(ventsettings_merge_df['t_0'])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('00:00:00','23:59:59'))
    day2= (
        (pd.to_datetime(ventsettings_merge_df['t_0'])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('24:00:00','47:59:59'))
    day3= (
        (pd.to_datetime(ventsettings_merge_df['t_0'])-pd.to_datetime(ventsettings_merge_df['charttime']))
           .between('48:00:00','71:59:59'))


    #0-1
    icu_O2count1= (
        ventsettings_merge_df.loc[day1 & ventsettings_merge_df['oxygentherapy']==1 ,:]
                  .groupby('icustay_id')['oxygentherapy']
                  .value_counts())
    icu_mechcount1= (
        ventsettings_merge_df.loc[day1 & ventsettings_merge_df['mechvent']==1 ,:]
                .groupby('icustay_id')['mechvent']
                .value_counts())

    #1-2
    icu_O2count2= (
        ventsettings_merge_df.loc[day2 & ventsettings_merge_df['oxygentherapy']==1 ,:]
                  .groupby('icustay_id')['oxygentherapy']
                  .value_counts())

    icu_mechcount2= (
        ventsettings_merge_df.loc[day2 & ventsettings_merge_df['mechvent']==1 ,:]
                .groupby('icustay_id')['mechvent']
                .value_counts())

    #2-3
    icu_O2count3= (
        ventsettings_merge_df.loc[day3 & ventsettings_merge_df['oxygentherapy']==1 ,:]
                  .groupby('icustay_id')['oxygentherapy']
                  .value_counts())

    icu_mechcount3= (
        ventsettings_merge_df.loc[day3 & ventsettings_merge_df['mechvent']==1 ,:]
                .groupby('icustay_id')['mechvent']
                .value_counts())

    #0-1
    icu_O2count_df1= pd.DataFrame(icu_O2count1)
    icu_O2count_df1.columns = ['O2_count1']
    icu_O2count_df1= icu_O2count_df1.reset_index()

    icu_mechcount_df1= pd.DataFrame(icu_mechcount1)
    icu_mechcount_df1.columns = ['mech_count1']
    icu_mechcount_df1= icu_mechcount_df1.reset_index()

    #1-2
    icu_O2count_df2= pd.DataFrame(icu_O2count2)
    icu_O2count_df2.columns = ['O2_count2']
    icu_O2count_df2= icu_O2count_df2.reset_index()

    icu_mechcount_df2= pd.DataFrame(icu_mechcount2)
    icu_mechcount_df2.columns = ['mech_count2']
    icu_mechcount_df2= icu_mechcount_df2.reset_index()

    #2-3
    icu_O2count_df3= pd.DataFrame(icu_O2count3)
    icu_O2count_df3.columns = ['O2_count_3']
    icu_O2count_df3= icu_O2count_df3.reset_index()

    icu_mechcount_df3= pd.DataFrame(icu_mechcount3)
    icu_mechcount_df3.columns = ['mech_count_3']
    icu_mechcount_df3= icu_mechcount_df3.reset_index()

    #merging all together
    icu_vent_status= pd.merge(icu_O2count_df1,icu_O2count_df2, how='outer')
    icu_vent_status= pd.merge(icu_vent_status,icu_O2count_df3,  how='outer')
    icu_vent_status= pd.merge(icu_vent_status,icu_mechcount_df1,  how='outer')
    icu_vent_status= pd.merge(icu_vent_status,icu_mechcount_df2,  how='outer')
    icu_vent_status= pd.merge(icu_vent_status,icu_mechcount_df3,  how='outer')
    
    return(icu_vent_status.drop(['oxygentherapy','mechvent'], axis=1))
icu_vent_status= vent_count(final_pt_df2,ventsettings_df)

In [10]:
ventcategory_df = vent_categorization(final_pt_df2, ventsettings_df)
ventcount_df = vent_count(final_pt_df2,ventsettings_df)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

time: 1.76 s


In [39]:
pd.DataFrame(ventsettings_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date))
pd.DataFrame(ventcategory_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date))
pd.DataFrame(ventcount_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcount_df.csv' %(date))

time: 5.56 s


time: 6 s


In [28]:
#adding an identification label column and merging them into 1 df. 
epi_dose_df['label']='epinephrine'
norepi_dose_df['label']='norepinephrine'
dopamine_dose_df['label']='dopamine'
dobutamine_dose_df['label']='dobutamine'
vasopressin_dose_df['label']='vasopressin'
phenylephrine_dose_df['label']='phenylephrine'
vaso_dose_df = pd.concat([epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df ])

time: 1.15 s


# saving as csv

In [29]:
date= '16082018'

pd.DataFrame(vaso_dose_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vaso_dose_df.csv' %(date))
pd.DataFrame(ventsettings_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date))
pd.DataFrame(ventcategory_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date))
pd.DataFrame(ventcount_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcount_df.csv' %(date))

pd.DataFrame(echodata_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_echodata_df.csv' %(date))
pd.DataFrame(weightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_weightfirstday_df.csv' %(date))
pd.DataFrame(heightfirstday_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_heightfirstday_df.csv' %(date))
pd.DataFrame(labs_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_labs_all_nosummary_df.csv' %(date))
pd.DataFrame(vitals_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date))
pd.DataFrame(gcsall_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcsall_df.csv' %(date))
pd.DataFrame(rrt_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_all_df.csv' %(date))
pd.DataFrame(urine_output_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_urine_output_all_df.csv' %(date))
pd.DataFrame(uti_all_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_uti_all_df.csv' %(date))
pd.DataFrame(bg_all_nosummary_df).to_csv(
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_bg_all_nosummary_df.csv' %(date))

time: 2min 21s


time: 5.08 s


#### ventilator data qc

In [55]:
#icustay_id: 267059.0  test case
#t_0= 2182-11-12
#t_0+72 = 2182-11-15
#t_end_consec = 2183-11-15

#vaso_dose_df.loc[vaso_dose_df.loc[:,'icustay_id']==267059.0,:].sort_values('starttime')
vaso_dose_df.loc[vaso_dose_df.loc[:,'icustay_id']==267059.0,'starttime'].describe() #Timestamp('2182-11-19 20:55:00')
#Timestamp('2182-11-19 20:55:00')
#Timestamp('2183-01-15 04:00:00)

###takeaway: looks like ventilator data is in proper format where we are getting all instances of administration throughout a patient's icustay. 

count                     116
unique                    106
top       2183-01-15 04:00:00
freq                        3
first     2182-11-19 20:55:00
last      2183-01-15 04:00:00
Name: starttime, dtype: object

time: 8.8 ms


## updated here through 08/03/18

# clinical variable QC

In [103]:
lab_bg_overlap_list=list(labsFirstDayDf[labsFirstDayDf['subject_id'].isin(list(
    bgFirstDayDf['subject_id'].unique()))]['subject_id'].unique())

time: 22.1 ms


In [131]:
#bgFirstDayDf
#(labsFirstDayDf[labsFirstDayDf['subject_id'].isin(lab_bg_overlap_list)])
(labsFirstDayDf[labsFirstDayDf['subject_id']==99912])[['potassium_min','potassium_max','icustay_id','subject_id','hadm_id']]

,potassium_min,potassium_max,icustay_id,subject_id,hadm_id
61504,3.9,4.6,236183,99912,189380


time: 10.1 ms


In [130]:
#5-9-18
#(bgFirstDayDf[bgFirstDayDf['subject_id']==(lab_bg_overlap_list)])
#(bgFirstDayDf[bgFirstDayDf['subject_id']==4])[['bicarbonate','specimen','charttime']]
(bgFirstDayDf[bgFirstDayDf['subject_id']==99912])[['potassium','specimen','charttime','icustay_id','subject_id','hadm_id']]
#bgFirstDayDf
#99966 and 99912 subject id has potassium


#qc, checking overlapping categories between bgFirstDayDf and labsFirstDatDf
#subject_id has potassium 4.7 on 2191-08-23 21:38:00 in bgFirstDayDf 
#subject_id has potassium 4.4-4.9 in labsFirstDayDf. 


(ie.intime - interval '6' hour) AND (ie.intime + interval '1' day)


#labs: # -- This query pivots lab values taken in the first 24 hours of a patient's stay
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit.
#these are from labevents
#labs taken between 6 hours prior to admin and 1 day after admission. 

#bloodgas: # -- The aim of this query is to pivot entries related to blood gases and
# -- chemistry values which were found in LABEVENTS
#labs taken between 6 hours prior to admin and 1 day after admission. 

#ok so labs uses itemid of either 50822 & 50971 where as bloodgas only uses 50822. 
#50822 is postassium, whole blood (category bloodgas) while 50971 is potassium in blood (category chemistry)

,potassium,specimen,charttime,icustay_id,subject_id,hadm_id
243343,4.4,None,2153-04-13 13:37:00,236183,99912,189380
243344,NaN,None,2153-04-13 16:26:00,236183,99912,189380
243345,NaN,None,2153-04-13 16:50:00,236183,99912,189380
243346,NaN,None,2153-04-13 18:36:00,236183,99912,189380
243347,NaN,None,2153-04-13 20:53:00,236183,99912,189380
243348,NaN,None,2153-04-13 21:54:00,236183,99912,189380
243349,NaN,None,2153-04-14 00:20:00,236183,99912,189380
243350,NaN,None,2153-04-14 01:50:00,236183,99912,189380
243351,NaN,None,2153-04-14 04:32:00,236183,99912,189380
243352,NaN,None,2153-04-14 06:38:00,236183,99912,189380


time: 19.2 ms
